In [2]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem
from sympy import Rational
import dill
dill.settings['recurse'] = True
from sympy import Rational as R

In [3]:
argyris_basis = symfem.create_element("triangle", "Argyris", 5)
argyris_basis.get_basis_functions()[18] *= -1
argyris_basis.get_basis_functions()[20] *= -1
argyris_basis = argyris_basis.get_basis_functions()

# Necessary symbols

In [4]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")

n1, n2, n3 = np.array(sympy.symbols("nx_1:4 ny_1:4")).reshape(2, 3).T
t1, t2, t3 = np.array(sympy.symbols("tx_1:4 ty_1:4")).reshape(2, 3).T

THETA = sympy.Matrix(np.array(sympy.symbols("H(1:4)1:4")).reshape(3, 3).tolist())

J = sympy.Matrix(np.array(sympy.symbols("J(1:3)1:3")).reshape(2, 2).tolist())

B1 = sympy.Matrix(np.array(sympy.symbols("BI_(1:3)1:3")).reshape(2, 2).tolist())
B2 = sympy.Matrix(np.array(sympy.symbols("BII_(1:3)1:3")).reshape(2, 2).tolist())
B3 = sympy.Matrix(np.array(sympy.symbols("BIII_(1:3)1:3")).reshape(2, 2).tolist())

DN_B1 = sympy.Matrix(np.array(sympy.symbols("DNBI_(1:3)1:3")).reshape(2, 2).tolist())
DN_B2 = sympy.Matrix(np.array(sympy.symbols("DNBII_(1:3)1:3")).reshape(2, 2).tolist())
DN_B3 = sympy.Matrix(np.array(sympy.symbols("DNBIII_(1:3)1:3")).reshape(2, 2).tolist())

tau_1 = sympy.Matrix([n1[0]**2, 2*n1[0]*n1[1], n1[1]**2])
tau_2 = sympy.Matrix([n2[0]**2, 2*n2[0]*n2[1], n2[1]**2])
tau_3 = sympy.Matrix([n3[0]**2, 2*n3[0]*n3[1], n3[1]**2])

l1, l2, l3 = sympy.symbols("l1:4")

# Functions

In [5]:
## Function values at the nodes
F_1 = argyris_basis[0] + 15/(8*l2)*B2.row(0)[1]*argyris_basis[19] + \
                              - 15/(8*l3)*B3.row(0)[1]*argyris_basis[20]

F_2 = argyris_basis[6] - 15/(8*l1)*B1.row(0)[1]*argyris_basis[18] + \
                              + 15/(8*l3)*B3.row(0)[1]*argyris_basis[20]

F_3 = argyris_basis[12] + 15/(8*l1)*B1.row(0)[1]*argyris_basis[18] + \
                               - 15/(8*l2)*B2.row(0)[1]*argyris_basis[19]

In [6]:
## Function derivative wit respect to x
DX_1 = J.inv().T.row(0)[0]*argyris_basis[1] + \
       J.inv().T.row(1)[0]*argyris_basis[2] + \
       - Rational(7, 16)*t2[0]*B2.row(0)[1]*argyris_basis[19] + \
       - Rational(7, 16)*t3[0]*B3.row(0)[1]*argyris_basis[20]

DX_2 = J.inv().T.row(0)[0]*argyris_basis[7] + \
       J.inv().T.row(1)[0]*argyris_basis[8] + \
       - Rational(7, 16)*t1[0]*B1.row(0)[1]*argyris_basis[18] + \
       - Rational(7, 16)*t3[0]*B3.row(0)[1]*argyris_basis[20]

DX_3 = J.inv().T.row(0)[0]*argyris_basis[13] + \
       J.inv().T.row(1)[0]*argyris_basis[14] + \
       - Rational(7, 16)*t1[0]*B1.row(0)[1]*argyris_basis[18] + \
       - Rational(7, 16)*t2[0]*B2.row(0)[1]*argyris_basis[19]

In [7]:
## Function derivative wit respect to y
DY_1 = J.inv().T.row(0)[1]*argyris_basis[1] + \
       J.inv().T.row(1)[1]*argyris_basis[2] + \
       - Rational(7, 16)*t2[1]*B2.row(0)[1]*argyris_basis[19] + \
       - Rational(7, 16)*t3[1]*B3.row(0)[1]*argyris_basis[20]

DY_2 = J.inv().T.row(0)[1]*argyris_basis[7] + \
       J.inv().T.row(1)[1]*argyris_basis[8] + \
       - Rational(7, 16)*t1[1]*B1.row(0)[1]*argyris_basis[18] + \
       - Rational(7, 16)*t3[1]*B3.row(0)[1]*argyris_basis[20]

DY_3 = J.inv().T.row(0)[1]*argyris_basis[13] + \
       J.inv().T.row(1)[1]*argyris_basis[14] + \
       - Rational(7, 16)*t1[1]*B1.row(0)[1]*argyris_basis[18] + \
       - Rational(7, 16)*t2[1]*B2.row(0)[1]*argyris_basis[19]

In [8]:
## Function second derivative wit respect to x
DXX_1 = THETA.inv().row(0)[0]*argyris_basis[3] + \
        THETA.inv().row(1)[0]*argyris_basis[4] + \
        THETA.inv().row(2)[0]*argyris_basis[5] + \
        + l2*Rational(1, 32)*tau_2[0]*B2.row(0)[1]*argyris_basis[19] + \
        - l3*Rational(1, 32)*tau_3[0]*B3.row(0)[1]*argyris_basis[20]

DXX_2 = THETA.inv().row(0)[0]*argyris_basis[9] + \
        THETA.inv().row(1)[0]*argyris_basis[10] + \
        THETA.inv().row(2)[0]*argyris_basis[11] + \
        - l1*Rational(1, 32)*tau_1[0]*B1.row(0)[1]*argyris_basis[18] + \
        + l3*Rational(1, 32)*tau_3[0]*B3.row(0)[1]*argyris_basis[20]

DXX_3 = THETA.inv().row(0)[0]*argyris_basis[15] + \
        THETA.inv().row(1)[0]*argyris_basis[16] + \
        THETA.inv().row(2)[0]*argyris_basis[17] + \
        + l1*Rational(1, 32)*tau_1[0]*B1.row(0)[1]*argyris_basis[18] + \
        - l2*Rational(1, 32)*tau_2[0]*B2.row(0)[1]*argyris_basis[19]

In [9]:
## Function second derivative wit respect to xy
DXY_1 = THETA.inv().row(0)[1]*argyris_basis[3] + \
        THETA.inv().row(1)[1]*argyris_basis[4] + \
        THETA.inv().row(2)[1]*argyris_basis[5] + \
        + l2*Rational(1, 32)*tau_2[1]*B2.row(0)[1]*argyris_basis[19] + \
        - l3*Rational(1, 32)*tau_3[1]*B3.row(0)[1]*argyris_basis[20]

DXY_2 = THETA.inv().row(0)[1]*argyris_basis[9] + \
        THETA.inv().row(1)[1]*argyris_basis[10] + \
        THETA.inv().row(2)[1]*argyris_basis[11] + \
        - l1*Rational(1, 32)*tau_1[1]*B1.row(0)[1]*argyris_basis[18] + \
        + l3*Rational(1, 32)*tau_3[1]*B3.row(0)[1]*argyris_basis[20]

DXY_3 = THETA.inv().row(0)[1]*argyris_basis[15] + \
        THETA.inv().row(1)[1]*argyris_basis[16] + \
        THETA.inv().row(2)[1]*argyris_basis[17] + \
        + l1*Rational(1, 32)*tau_1[1]*B1.row(0)[1]*argyris_basis[18] + \
        - l2*Rational(1, 32)*tau_2[1]*B2.row(0)[1]*argyris_basis[19]

In [10]:
## Function second derivative wit respect to y
DYY_1 = THETA.inv().row(0)[2]*argyris_basis[3] + \
        THETA.inv().row(1)[2]*argyris_basis[4] + \
        THETA.inv().row(2)[2]*argyris_basis[5] + \
        + l2*Rational(1, 32)*tau_2[2]*B2.row(0)[1]*argyris_basis[19] + \
        - l3*Rational(1, 32)*tau_3[2]*B3.row(0)[1]*argyris_basis[20]

DYY_2 = THETA.inv().row(0)[2]*argyris_basis[9] + \
        THETA.inv().row(1)[2]*argyris_basis[10] + \
        THETA.inv().row(2)[2]*argyris_basis[11] + \
        - l1*Rational(1, 32)*tau_1[2]*B1.row(0)[1]*argyris_basis[18] + \
        + l3*Rational(1, 32)*tau_3[2]*B3.row(0)[1]*argyris_basis[20]

DYY_3 = THETA.inv().row(0)[2]*argyris_basis[15] + \
        THETA.inv().row(1)[2]*argyris_basis[16] + \
        THETA.inv().row(2)[2]*argyris_basis[17] + \
        + l1*Rational(1, 32)*tau_1[2]*B1.row(0)[1]*argyris_basis[18] + \
        - l2*Rational(1, 32)*tau_2[2]*B2.row(0)[1]*argyris_basis[19]

In [11]:
DN_1 = +DN_B1.row(0)[0]*argyris_basis[18]
DN_2 = +DN_B2.row(0)[0]*argyris_basis[19]
DN_3 = +DN_B3.row(0)[0]*argyris_basis[20]

In [12]:
ALL_BASIS = \
        [
                F_1, F_2, F_3,
                DX_1, DX_2, DX_3,
                DY_1, DY_2, DY_3,
                DXX_1, DXX_2, DXX_3,
                DXY_1, DXY_2, DXY_3,
                DYY_1, DYY_2, DYY_3,
                DN_1, DN_2, DN_3
        ]

In [13]:
syms = [
        *n1, *n2, *n3, 
        *t1, *t2, *t3,
        l1, l2, l3,
        *np.array(J.tolist()).flatten(), 
        *np.array(THETA.tolist()).flatten(),
        *np.array(B1.tolist()).flatten(),
        *np.array(B2.tolist()).flatten(),
        *np.array(B3.tolist()).flatten(),
        *np.array(DN_B1.tolist()).flatten(),
        *np.array(DN_B2.tolist()).flatten(),
        *np.array(DN_B3.tolist()).flatten(),
       ]

# New way

In [15]:
N = len(argyris_basis)

In [17]:
result_b = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]


for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):

    first = argyris_basis[idx]
    second = argyris_basis[jdx]
    
    u_x = first.diff(x)
    u_y = first.diff(y)
    
    u_xx = u_x.diff(x)
    u_xy = u_x.diff(y)
    
    u_yy = u_y.diff(y)
    
    v_x = second.diff(x)
    v_y = second.diff(y)
    
    v_xx = v_x.diff(x)
    v_xy = v_x.diff(y)
    
    v_yy = v_y.diff(y)
    
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy

    sym_int_B = sympy.integrate(A + 2*B + C, (y, 0, 1-x), (x, 0, 1))
    sym_int_b = sympy.integrate(A + 2*b + C, (y, 0, 1-x), (x, 0, 1))
    
    result_B[idx][jdx] = sym_int_B
    result_b[idx][jdx] = sym_int_b

100%|██████████| 441/441 [09:29<00:00,  1.29s/it]


In [18]:
result_A = []

for basis in tqdm(argyris_basis):

    sym_int = sympy.integrate(basis, (y, 0, 1-x), (x, 0, 1))
    
    result_A.append(sym_int)
result_A = sympy.Matrix(result_A)

100%|██████████| 21/21 [00:03<00:00,  6.17it/s]


# Symbolic stuff

In [19]:
V_c = sympy.diag(1, J.inv().T, THETA.inv(), 1, J.inv().T, THETA.inv(), 1, J.inv().T, THETA.inv(), B1, B2, B3)

In [25]:
E = np.zeros((21, 24), dtype=int)

for i in range(21):
    for j in range(24):
        if (i <= 18) and (j <= 18) and (i == j):
            E[i][j] = 1
        elif (i == 19) and (j == 20):
            E[i][j] = 1
        elif (i == 20) and (j == 22):
            E[i][j] = 1
E = sympy.Matrix(E)

In [26]:
D = sympy.Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, -15/(8*l1), -R(7,16)*t1[0], -R(7,16)*t1[1], -l1/32*tau_1[0], -l1/32*tau_1[1], -l1/32*tau_1[2], +15/(8*l1), -R(7,16)*t1[0], -R(7,16)*t1[1], +l1/32*tau_1[0], +l1/32*tau_1[1], +l1/32*tau_1[2], 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[+15/(8*l2), -R(7,16)*t2[0], -R(7,16)*t2[1], +l2/32*tau_2[0], +l2/32*tau_2[1], +l2/32*tau_2[2], 0, 0, 0, 0, 0, 0, -15/(8*l2), -R(7,16)*t2[0], -R(7,16)*t2[1], -l2/32*tau_2[0], -l2/32*tau_2[1], -l2/32*tau_2[2], 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[-15/(8*l3), -R(7,16)*t3[0], -R(7,16)*t3[1], -l3/32*tau_3[0], -l3/32*tau_3[1], -l3/32*tau_3[2], +15/(8*l3), -R(7,16)*t3[0], -R(7,16)*t3[1], +l3/32*tau_3[0], +l3/32*tau_3[1], +l3/32*tau_3[2], 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

In [77]:
M = (E@V_c@D).T

In [78]:
M[18*21 + 18] = DN_B1[0]
M[19*21 + 19] = DN_B2[0]
M[20*21 + 20] = DN_B3[0]

In [80]:
result_b = sympy.Matrix(result_b)
result_B = sympy.Matrix(result_B)

In [81]:
FUNC_b = sympy.lambdify(syms, M@result_b@M.T)
FUNC_B = sympy.lambdify(syms, M@result_B@M.T)

In [47]:
dill.dump(FUNC_b, open("../calculations/FUNC_b", "wb"))
dill.dump(FUNC_B, open("../calculations/FUNC_B", "wb"))

In [48]:
FUNC_A = sympy.lambdify(syms, M@result_A)
dill.dump(FUNC_A, open("../calculations/FUNC_A", "wb"))

In [49]:
U = (1-(x**2 + y**2))**2/64

In [52]:
sympy.diff(U, x, x, x, x)+sympy.diff(U, y, y, y, y) + 2*sympy.diff(U, x, x, y, y)

1